<h1>Roll Balance and Chassis Equations


Sources:
RCVD...

In [1]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

<h3>Initial Ride Rate Calculations</h3>

Non ground effect Indy Cars ran 95-120cpm(1.58-2.0000Hz ) which seems like a good place to start - RCVD 16.2

To insure initial understeer, calculate the Total Lateral
Load Transfer Distribution (TLLTD) to be 5% more than
the weight distribution at the front - RCVD table 16.5

In [2]:
vehicle_mass = 250 + 80 #kg
rear_mass_ratio = 0.6
rear_mass = vehicle_mass*rear_mass_ratio
front_mass = vehicle_mass*(1-rear_mass_ratio)

def ride_rate(natural_frequency, mass):
    return mass * natural_frequency**2


target_frequency = 2.0 * 2*np.pi # rad/s
rear_wheel_ride_rate = ride_rate(target_frequency, rear_mass/2)
print(f'The ideal individual rear wheel ride rate for the target frequency is {rear_wheel_ride_rate*1e-3 :.2f} N/mm; {5.7101471627692*rear_wheel_ride_rate*1e-3 :.2f} lb/in\n')



front_wheel_ride_rate = ride_rate(target_frequency, front_mass/2)
print(f'The ideal individual front wheel ride rate for the target frequency is {front_wheel_ride_rate*1e-3 :.2f} N/mm; {5.7101471627692*front_wheel_ride_rate*1e-3 :.2f} lb/in')


front_natural_frequency = (rear_wheel_ride_rate/(front_mass/2))**(1/2)
print(f'The front natural frequency using the same spring rates as the rear is {front_natural_frequency/(2*np.pi) :.2f} hz')


The ideal individual rear wheel ride rate for the target frequency is 15.63 N/mm; 89.27 lb/in

The ideal individual front wheel ride rate for the target frequency is 10.42 N/mm; 59.51 lb/in
The front natural frequency using the same spring rates as the rear is 2.45 hz


<h3>Instant Rear Roll Center Placement</h3> 

In [3]:
def roll_center_height(desired_wheel_deflection, track, wheel_rate, cg_height, axle_mass, lateral_acceleration):
    return cg_height - desired_wheel_deflection*track*wheel_rate/(2*axle_mass*lateral_acceleration)

total_wheel_travel = 35e-3 #m
roll_travel_ratio = 0.8

track_width = 1300e-3 #m
wheelbase = 1530e-3 #m

center_of_mass_height = 220*1e-3 + total_wheel_travel #m
g = 9.81 #m/s^2
maximum_lateral_acceleration = 1.3*g#m/s^2

roll_axis_inclination = 2 #degrees

roll_wheel_deflection = total_wheel_travel*(0.8)
rear_roll_center_height = roll_center_height(roll_wheel_deflection, track_width, rear_wheel_ride_rate, center_of_mass_height, vehicle_mass/2, maximum_lateral_acceleration)
print(f'The roll center height needed to meet target rear wheel travel in roll is {rear_roll_center_height*1e+3 :.2f} mm')



front_roll_center_height = rear_roll_center_height - np.tan(np.radians(roll_axis_inclination))*wheelbase
print(f'The front roll center height needed to give a roll axis inclination of {roll_axis_inclination} degrees is {front_roll_center_height*1e+3 :.2f} mm')


maximum_body_roll_angle = np.degrees(np.atan(2*roll_wheel_deflection)/track_width)
print(f'\nThe body roll angle at a lateral acceleration of {maximum_lateral_acceleration :.2f} is {maximum_body_roll_angle :.2f} degrees')

rear_roll_gradient = maximum_body_roll_angle / maximum_lateral_acceleration  * g
print(f'The rear roll gradient from the chose parameters is {rear_roll_gradient :.2f} degrees/g')


The roll center height needed to meet target rear wheel travel in roll is 119.78 mm
The front roll center height needed to give a roll axis inclination of 2 degrees is 66.35 mm

The body roll angle at a lateral acceleration of 12.75 is 2.47 degrees
The rear roll gradient from the chose parameters is 1.90 degrees/g


<h2> Symbolically set up load transfer equations

In [4]:
import sympy
total_load_transfer, load_transfer_front, load_transfer_rear, roll_stiffness_front, roll_stiffness_rear, chassis_stiffness, ac = sympy.symbols('LT LT_f LT_r K_f K_r K_ch a_c')
track_symb, center_of_mass_height_symb, RCH_f_symb, RCH_r_symb, mass_symb, mass_front_symb, mass_rear_symb = sympy.symbols('T H_cg RCH_f RCH_r m m_f m_r')

f_lt_ratio, f_roll_spring_ratio, r_mass_ratio, chassis_tuning_constant, lt_to_acceleration_ratio = sympy.symbols('η λ γ κ ξ')

phi_1, phi_2, front_inertial_moment, rear_inertial_moment = sympy.symbols('phi_1 phi_2 M_f M_r')

front_moment_balance = sympy.Eq(front_inertial_moment, roll_stiffness_front*phi_1 - chassis_stiffness*(phi_2-phi_1))
rear_moment_balance = sympy.Eq(rear_inertial_moment, roll_stiffness_rear*phi_2 + chassis_stiffness*(phi_2-phi_1))

print('Moment Balance Equations:')
display(front_moment_balance)
display(rear_moment_balance)

chassis_tuning_eq = sympy.Eq(chassis_tuning_constant**-1, (f_roll_spring_ratio-1)*(f_lt_ratio+1)/((f_lt_ratio-1)*(f_roll_spring_ratio+1))) #This inverse doesnt match my notes but it seems to give better results
print('\nChassis Tuning Constant Equation:')
display(chassis_tuning_eq)


roll_angles = sympy.solve([front_moment_balance, rear_moment_balance], [phi_1, phi_2])
load_transfer_equations = [sympy.Eq(roll_stiffness_front*roll_angles[phi_1]/track_symb, load_transfer_front), sympy.Eq(roll_stiffness_rear*roll_angles[phi_2]/track_symb, load_transfer_rear), chassis_tuning_eq]

print('\nLoad Transfer Expressions:')
display(*load_transfer_equations)


#total_load_transfer = 2*mass*ac*h/t

vals = [(roll_stiffness_front, f_roll_spring_ratio*roll_stiffness_rear), 
        
        (front_inertial_moment, mass_front_symb*ac*(center_of_mass_height_symb-RCH_f_symb)), 
        (rear_inertial_moment, mass_rear_symb*ac*(center_of_mass_height_symb-RCH_r_symb)), 


        (load_transfer_front, load_transfer_rear* f_lt_ratio), 
        (load_transfer_rear, total_load_transfer/(1+f_lt_ratio)),

        (mass_front_symb, (1-r_mass_ratio)*mass_symb), 
        (mass_rear_symb, r_mass_ratio*mass_symb),

        (total_load_transfer, lt_to_acceleration_ratio*ac)
        ]

nondim_equations = [expr.subs(vals).simplify() for expr in load_transfer_equations]

print('\nNondim Load Transfer Equations:')
display(*nondim_equations)


Moment Balance Equations:


Eq(M_f, -K_ch*(-phi_1 + phi_2) + K_f*phi_1)

Eq(M_r, K_ch*(-phi_1 + phi_2) + K_r*phi_2)


Chassis Tuning Constant Equation:


Eq(1/κ, (η + 1)*(λ - 1)/((η - 1)*(λ + 1)))

...

Eq(1/κ, (η + 1)*(λ - 1)/((η - 1)*(λ + 1)))


Nondim Load Transfer Equations:


Eq(a_c*η*ξ/(η + 1), -a_c*m*λ*(-K_ch*γ*(H_cg - RCH_r) + K_ch*(H_cg - RCH_f)*(γ - 1) + K_r*(H_cg - RCH_f)*(γ - 1))/(T*(K_ch*λ + K_ch + K_r*λ)))

Eq(a_c*ξ/(η + 1), a_c*m*(K_ch*γ*(H_cg - RCH_r) - K_ch*(H_cg - RCH_f)*(γ - 1) + K_r*γ*λ*(H_cg - RCH_r))/(T*(K_ch*λ + K_ch + K_r*λ)))

Eq(1/κ, (η + 1)*(λ - 1)/((η - 1)*(λ + 1)))

<h3> Front ARB and chassis stiffness </h3>

In [5]:
rear_spring_roll_stiffness = track_width**2 * rear_wheel_ride_rate / 2 
print(f'The rear roll rate is {rear_spring_roll_stiffness*2*np.pi/180 :.2f} Nm/deg')


variable_values = [
    (track_symb, track_width), 
    (center_of_mass_height_symb, center_of_mass_height), 
    (RCH_f_symb, front_roll_center_height), 
    (RCH_r_symb, rear_roll_center_height), 
    (mass_symb, vehicle_mass),
    (f_lt_ratio, 1.05), #slightly higher front load transfer is to give margin for understeer 
    (r_mass_ratio, rear_mass_ratio), 
    (roll_stiffness_rear, rear_spring_roll_stiffness),
    (chassis_tuning_constant, 0.8)  #ratio of how a change in roll stiffness balance affects the load transfer balance
]

print('\nExpressions for substitution:')
display(*[sympy.Eq(*equality) for equality in variable_values] )

numeric_equations = [eq.subs(variable_values).simplify() for eq in nondim_equations]
print('\nFinal System of Equations:')
display(*numeric_equations)  
chassis_stiffness, front_roll_stiffness_ratio, lt_to_ac_ratio  = sympy.solve(numeric_equations, [chassis_stiffness, f_roll_spring_ratio, lt_to_acceleration_ratio])[0]

print(f'\nchassis_stiffness: {chassis_stiffness*np.pi/180 :.2f} Nm/deg        front spring roll stiffness ratio: {front_roll_stiffness_ratio :.2f}        LT/ac: {lt_to_ac_ratio :.2f}')



front_roll_spring_rate = front_roll_stiffness_ratio*rear_spring_roll_stiffness
print(f'\nThe front roll rate which give a front load transfer ratio of {1.05} is {front_roll_spring_rate*2*np.pi/180 :.2f} Nm/deg')

front_roll_rate_from_ride = track_width**2 * front_wheel_ride_rate / 2 #TODO double check constants in equation
front_arb_contribution = (front_roll_spring_rate-front_roll_rate_from_ride)/front_roll_spring_rate
print(f'The front ARB contributes {front_arb_contribution*100 :.2f}% to the front roll stiffness')


### THIS IS A NUMERIC SOLUTION TO THE SAME EQUATIONS AS ABOVE, PROBABLY NOT NESSECARY 
#numeric_exprs = [(1-eq.rhs/eq.lhs).simplify() for eq in numeric_equations]
#print('\n the expressions for numeric evaluation:')
#display(*numeric_exprs)
#numeric_equation_functions = [sympy.Lambda((kch, f_roll_spring_ratio, lt_to_acceleration_ratio), expr) for expr in numeric_exprs]
#def f(x, function_list=numeric_equation_functions):
    #return [func(*x) for func in function_list]

#chassis_stiffness, front_roll_stiffness_ratio, lt_to_ac_ratio  = fsolve(f, [1000 * 180/np.pi, 1.05, 40])
#print(f'\nchassis_stiffness: {chassis_stiffness*np.pi/180 :.2f}        front roll stiffness ratio: {front_roll_stiffness_ratio :.2f}        LT/ac: {lt_to_ac_ratio :.2f}')




The rear roll rate is 461.13 Nm/deg

Expressions for substitution:


Eq(T, 1.3)

Eq(H_cg, 0.255)

Eq(RCH_f, 0.0663546241550562)

Eq(RCH_r, 0.11978340147743)

...


Final System of Equations:


Eq(0.51219512195122*a_c*ξ, 253.846153846154*a_c*λ*(0.156588109451519*K_ch + 996.822396201982)/(K_ch*λ + K_ch + 13210.2680987701*λ))

Eq(0.487804878048781*a_c*ξ, 253.846153846154*a_c*(0.156588109451519*K_ch + 1071.74851073214*λ)/(K_ch*λ + K_ch + 13210.2680987701*λ))

Eq(41.0*(λ - 1)/(λ + 1), 1.25)


chassis_stiffness: 1180.87 Nm/deg        front spring roll stiffness ratio: 1.06        LT/ac: 39.75

The front roll rate which give a front load transfer ratio of 1.05 is 490.13 Nm/deg
The front ARB contributes 37.28% to the front roll stiffness


<h3>Suspension Pickup Point Calculation</h3>



In [6]:
loaded_wheel_radius = 230e-3 #m
suspension_pickup_spacing = 250e-3 #m
minimum_lca_centerline_distance = 150e-3 #m         This is the minimum distance from the LCA chassis pickup to the centerline in order to adhere to rules

upright_pickup_offset = 50e-3 #m
kpi = np.radians(5) #rads

dynamic_camber = np.radians(8) #rads        This is a wild guess at camber to give maximum lateral acceleration


